<a href="https://colab.research.google.com/github/AyazMurtazin/DeepLearnPythonKPFU/blob/main/ML/Practical_work_2_Multilayer_perceptron_(MLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Практическая работа №2. Многослойный перцептрон (MLP)**



## **Задача 1: Подготовка данных**


In [ ]:

from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

# Загрузка данных
data = fetch_california_housing()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Проверка на пропущенные значения
print("Пропущенные значения:\n", df.isnull().sum())

# Обнаружение выбросов методом Isolation Forest
iso = IsolationForest(contamination=0.01, random_state=42)
outliers = iso.fit_predict(df.drop(columns='target'))
df_cleaned = df[outliers == 1]

# Масштабирование числовых признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cleaned.drop(columns='target'))
y = df_cleaned['target'].values

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)




## **Задача 2: Feature Engineering**


In [ ]:

from sklearn.preprocessing import PolynomialFeatures

# Генерация полиномиальных признаков (степень 2)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Проверка на мультиколлинеарность через корреляцию
corr_matrix = pd.DataFrame(X_train_poly).corr()
sns.heatmap(corr_matrix, cmap='coolwarm')
plt.title("Корреляционная матрица новых признаков")
plt.show()




## **Задача 3: Анализ корреляции и проверка гипотез**


In [ ]:

from scipy.stats import ttest_ind, shapiro

# Корреляционная матрица
sns.heatmap(df_cleaned.corr(), annot=True, cmap='coolwarm')
plt.title("Корреляция признаков")
plt.show()

# Проверка нормальности (Шапиро-Уилка) для одного признака
stat, p = shapiro(df_cleaned['AveRooms'])
print(f"Шапиро-Уилк для 'AveRooms': p-value = {p:.4f}")




## **Задача 4: Подготовка данных для MLP**


In [ ]:

# Повторное масштабирование с учетом многослойного перцептрона (если нужно)
# В нашем случае уже применён StandardScaler на табличные данные
X_mlp_train, X_mlp_test = X_train_poly, X_test_poly  # Используем расширенные признаки




## **Задача 5: Построение и анализ архитектуры MLP**


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import time

def build_model(layers):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_mlp_train.shape[1],)))
    for _ in range(layers - 2):
        model.add(Dense(64, activation='relu'))
    model.add(Dense(1))  # выходной слой для регрессии
    model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])
    return model

# Сравнение 3 vs 5 слоёв
for depth in [3, 5]:
    model = build_model(depth)
    start = time.time()
    history = model.fit(X_mlp_train, y_train, validation_split=0.2, epochs=20, verbose=0)
    duration = time.time() - start
    print(f"Model with {depth} layers: MAE = {min(history.history['val_mae']):.4f}, Time = {duration:.2f}s")




## **Задача 6: Методы оптимизации и анализ градиентов**


In [ ]:

import torch
import torch.nn as nn

# Пример градиента для MSE
x = torch.tensor([1.0, 2.0], requires_grad=True)
y_true = torch.tensor([1.5])
model = nn.Linear(2, 1)
criterion = nn.MSELoss()

y_pred = model(x)
loss = criterion(y_pred, y_true)
loss.backward()
print(f"Градиенты: {x.grad}")




## **Задача 7: Регуляризация и борьба с переобучением**


In [ ]:

from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout, BatchNormalization

model_reg = Sequential([
    Dense(64, activation='relu', kernel_regularizer=l2(0.01), input_shape=(X_mlp_train.shape[1],)),
    Dropout(0.3),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1)
])
model_reg.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_reg.fit(X_mlp_train, y_train, epochs=20, validation_split=0.2, verbose=0)




## **Задача 8: Оценка качества и интерпретация**


In [ ]:

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import shap

# Оценка модели
y_pred = model_reg.predict(X_mlp_test)
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))

# Интерпретация SHAP
explainer = shap.Explainer(model_reg, X_mlp_train)
shap_values = explainer(X_mlp_test[:100])
shap.plots.beeswarm(shap_values)




## **Задача 9: Продвинутые техники и оптимизация**


In [ ]:
!pip install -q "optuna>=3.0.0"

In [ ]:
import optuna

def objective(trial):
    n_layers = trial.suggest_int('hidden_layers', 2, 4)
    units = trial.suggest_categorical('units', [32, 64, 128])
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)

    model = Sequential()
    model.add(Dense(units, activation='relu', input_shape=(X_mlp_train.shape[1],)))
    for _ in range(n_layers - 1):
        model.add(Dense(units, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    model.fit(X_mlp_train, y_train, validation_split=0.2, epochs=10, verbose=0)
    loss, mae = model.evaluate(X_mlp_test, y_test, verbose=0)
    return mae

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
print("Лучшие параметры:", study.best_params)




## **Задача 10: Анализ результатов и выбор модели**


In [ ]:
# Обучение и предсказания для модели с 3 скрытыми слоями
model_3 = build_model(3)
model_3.fit(X_mlp_train, y_train, epochs=10, verbose=0)
y_pred_3 = model_3.predict(X_mlp_test)

# Обучение и предсказания для модели с 5 скрытыми слоями
model_5 = build_model(5)
model_5.fit(X_mlp_train, y_train, epochs=10, verbose=0)
y_pred_5 = model_5.predict(X_mlp_test)

# Предсказания модели с регуляризацией (предположим, она обучена заранее как model_reg)
y_pred_reg = model_reg.predict(X_mlp_test)

# Оценка качества моделей
mae_3 = mean_absolute_error(y_test, y_pred_3)
mse_3 = mean_squared_error(y_test, y_pred_3)

mae_5 = mean_absolute_error(y_test, y_pred_5)
mse_5 = mean_squared_error(y_test, y_pred_5)

mae_reg = mean_absolute_error(y_test, y_pred_reg)
mse_reg = mean_squared_error(y_test, y_pred_reg)

# Сравнение результатов
results = {
    'MLP (3 hidden layers)': {'MAE': mae_3, 'MSE': mse_3},
    'MLP (5 hidden layers)': {'MAE': mae_5, 'MSE': mse_5},
    'MLP с регуляризацией': {'MAE': mae_reg, 'MSE': mse_reg}
}

# Вывод таблицы
results_df = pd.DataFrame(results).T
print("Сравнение моделей:")
print(results_df)

# Лучшая модель по MAE
best_model_name = results_df['MAE'].idxmin()
print(f"\nЛучшая модель по MAE: {best_model_name}")
